#### Save this file within the same folder as your workpaper (where you have your journal batches on separate tabs)

#### Open this file in Jupyter Notebook and run the script below

#### Import Libraries

In [ ]:
import os
import pandas as pd

##### Read all the tabs within the excel file

In [ ]:
#Set f equals your wp name
f = 'OCT22_LegalAccrual.xlsx'

In [ ]:
#Pass f in as the first parameter -

sheets_dict = pd.read_excel(f, sheet_name=None) 

#### Create a empty list to store journal header name (will pass it as the header for the final journal batch)

In [ ]:
header =[]

#Read one of the tabs that have the journal batch template -"Batch_Invoice_(1)"
#header is in row 12, therefore, the 3rd parameter is - header=11 - as Python start counting from zero
#if on your file the header is in row 11, make sure to update parameter header=10, etc. 

Batch1 = pd.read_excel(f, "Batch_Invoice_(1)", header=11)
header = list(Batch1.columns.values)
print(header)

#### Create a empty list to store all the journal batch tabs (make sure those tabs have the same naming convention so we can set criteria to iterate and find those tabs within the file)

In [ ]:
all_sheets = []

In [ ]:
for name, value in sheets_dict.items():
    if name and "Batch" in name: #all the tabs have the name "Batch" at front in my case here

        # Delete the first 11 rows - Remember Python start counting from zero
        value = value.drop(value.index[range(11)])

        all_sheets.append(value)

#### Check if the total number printed below matches with number of tabs you want to combine. 

In [ ]:
print(len(all_sheets)) #Total number of tabs that meet the search criteria

##### Combine all the tabs into a single file 

In [ ]:
full_journal = pd.concat(all_sheets) 

In [ ]:
full_journal.head() #Print the header of the file

In [ ]:
full_journal.columns = header #Pass the header stored earlier into the new file as the header

In [ ]:
full_journal.head() #Print the new header

#### Set the conditions to drop lines (in many cases, it will be Debit Amount zero, in this case it is the NAN value in Company column)

In [ ]:
full_journal = full_journal.dropna(subset=['*Company [..]']) #Drop the lines with the none value in Company

In [ ]:
full_journal= full_journal[full_journal['**Entered Debit'] != 0] #Drop the lines with zero value 

In [ ]:
full_journal = full_journal.dropna(subset=['**Entered Debit']) #Drop the lines with the none value in Entered Debit

#### Validate the total debit amount

In [ ]:
total = full_journal['**Entered Debit'].sum()
print(total)

#### Validate the total amount per entity

In [ ]:
entity_total = full_journal.groupby('*Company [..]')['**Entered Debit'].sum()
print(entity_total)

#### Validate the total amount per GL account

In [ ]:
account_total = full_journal.groupby('*Account [..]')['**Entered Debit'].sum()
print(account_total)

#### Export the new file as excel 

In [ ]:
output = "journals_legal_oct.xlsx"

In [ ]:
full_journal.to_excel(output, index=False)

In [ ]:
from openpyxl import load_workbook

wb = load_workbook(output)
target = wb['Sheet1']
wb.copy_worksheet(target)
wb.save(output)

In [ ]:
sheetcopy = pd.read_excel(output,"Sheet1 Copy")

In [ ]:
for i, row in sheetcopy.iterrows():
    if sheetcopy.loc[i,'*Account [..]'] == 66108:
        sheetcopy.loc[i,'*Account [..]'] = 23111
    elif sheetcopy.loc[i,'*Account [..]'] == 62110:
        sheetcopy.loc[i,'*Account [..]'] = 23106
    else:
        sheetcopy.loc[i,'*Account [..]'] = 23102

account_total2 = sheetcopy.groupby('*Account [..]')['**Entered Debit'].sum()
print(account_total2)

In [ ]:
sheetcopy['*Department [..]'] = sheetcopy['*Department [..]'].replace('000')

In [ ]:
for i, row in sheetcopy.iterrows():
    if sheetcopy.loc[i,'*Product [..]'] == 0:
        sheetcopy.loc[i,'*Product [..]'] = '000'
    else:
        pass

In [ ]:
sheetcopy.loc[:, '*Project [..]'] = '000'

In [ ]:
for i, row in sheetcopy.iterrows():
    if sheetcopy.loc[i, '*Future 1 [..]'] == 0:
        sheetcopy.loc[i,'*Future 1 [..]'] = '000'
    else:
        pass

In [ ]:
for i, row in sheetcopy.iterrows():
    if sheetcopy.loc[i, '*Intercompany [..]'] == 0:
        sheetcopy.loc[i,'*Intercompany [..]'] = '000'
    else:
        pass

In [ ]:
for i, row in sheetcopy.iterrows():
    if sheetcopy.loc[i, '*Future 2 [..]'] == 0:
        sheetcopy.loc[i,'*Future 2 [..]'] = '000'
    else:
        pass

In [ ]:
sheetcopy.loc[:, '*Department [..]'] = '000'

In [ ]:
sheetcopy.loc[:, '*Location [..]'] = '000'

In [ ]:
sheetcopy['**Entered Credit'] = sheetcopy['**Entered Credit'].fillna(sheetcopy['**Entered Debit'])

In [ ]:
sheetcopy.loc[:, '**Entered Debit'] = ""

In [ ]:
frames = [full_journal, sheetcopy]

result = pd.concat(frames)

result.to_excel(output, index=True)